In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
from os import path, listdir
import  cv2
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train = pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')
test = pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')
# train = pd.read_csv('input/train.csv')
# test = pd.read_csv('input/test.csv')

In [ ]:
# checking data
train.head(5)

In [ ]:
# train_ID = train['image_id']
# test_ID = test['image_id']

In [ ]:
# train.drop("image_id", axis = 1, inplace = True)
# test.drop("image_id", axis = 1, inplace = True)

In [ ]:
'''
0 healthy
1 multiple_diseases
2 rust
3 scab
'''  
type(train)
train=train[train.multiple_diseases==0]

In [ ]:
# y_train = train.values

In [ ]:
#  image folder path
# path1 = "../input/plant-pathology-2020-fgvc7/images"


In [ ]:
%%time
# path1 = "inputimages"
path1 = "../input/plant-pathology-2020-fgvc7/images"

# imagePaths = [path.join(path1, f) for f in listdir(path1)]
train_data = []
train_label= []
for index, row in train.iterrows():
    print(index,row['image_id'])
    image = cv2.imread(path1+"/"+row['image_id']+".jpg")
    image = cv2.resize(image, (227,227))
    train_data.append(image)
    train_label.append([row[1],row[3],row[4]])

In [ ]:
print(len(train_label))
print(len(train_data))
train_label[0]
# train_data[0]
arr=[[4,5,5]]
np.array(arr)

In [36]:
from sklearn.model_selection import train_test_split

train_data_np = np.array(train_data)
train_label_np = np.array(train_label)
(trainX, testX, trainY, testY) = train_test_split(train_data_np, train_label_np,
	test_size=0.3, stratify=train_label_np, random_state=42)

In [37]:
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
valAug = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [38]:
from keras.applications import VGG16
from keras.layers import Input
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(227, 227, 3)))

In [39]:
from keras.layers.pooling import AveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.3)(headModel)
headModel = Dense(3, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
	layer.trainable = False
print("[INFO] compiling model...")
opt = Adam(lr=1e-4)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [ ]:
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=32),
	steps_per_epoch=len(trainX) // 32,
	validation_data=valAug.flow(testX, testY),
	validation_steps=len(testX) // 32,
	epochs=40,
    shuffle=True)

Epoch 1/1
12/37 [========>.....................] - ETA: 10s - loss: 3.1507 - accuracy: 0.5512

In [ ]:
model.save("trained_model_7_39.h5")

In [ ]:
# path1 = "../input/plant-pathology-2020-fgvc7/images"
# for index, row in test.iterrows():
#     print(index,row['image_id'])
#     image = cv2.imread(path1+"/"+row['image_id']+".jpg")
#     image = cv2.resize(image, (227,227))
predictions = model.predict(testX)

In [ ]:
predictions

In [ ]:
path1 = "../input/plant-pathology-2020-fgvc7/images"
test_data=[]
for index, row in test.iterrows():
    print(index,row['image_id'])
    image = cv2.imread(path1+"/"+row['image_id']+".jpg")
    image = cv2.resize(image, (227,227))
    test_data.append(image)

In [ ]:
test_data_np=np.array(test_data)

In [ ]:
test_predict = model.predict(test_data_np)

In [ ]:
numpy_data = np.array([[1, 2], [3, 4]])
category_df = pd.DataFrame(data=test_predict, columns=['healthy', 'rust', 'scab'])


In [ ]:
# df=category_df
# category_df = df.eq(df.where(df != 0).max(1), axis=0).astype(int)

In [ ]:
subm = pd.concat([test,category_df],axis=1)

In [ ]:
subm.insert(2, 'multiple_diseases', 0)

In [ ]:
subm

In [ ]:
subm.to_csv('submission3.csv',index=False)